# Time to make a new getImage function
- Must have subroutine to more clearly select datasetRef

In [1]:
# where = "exposure.science_program=run_number"
# refs = list(butler.registry.queryDatasets("raw", where=where,  bind={"run_number": run_number}, detector=detector))

In [2]:
import sys
sys.path.append('/home/s/seanmacb/Camera/pkgs')
sys.path.append('/home/s/seanmacb/Camera')
sys.path.append('/home/s/seanmacb/Camera/Run6Comparisons/')
import run_six_utils as rsu
import lsst.afw.display as afwDisplay
import lsst.daf.butler as daf_butler
afwDisplay.setDefaultBackend('matplotlib') 
import matplotlib.pyplot as plt
import numpy as np
import myutils_ar.myutils as myu
import eo_pipe

In [3]:
def getRefList(registry,kwargs):
    '''
    Function to get a list of datasetRefs, given the kwargs and a registryShim
    '''
    return list(registry.queryDatasets(**kwargs))
    
def makeButler(repo,collections):
    '''
    A function to return a butler object, given the repo and collections specified
    '''
    return daf_butler.Butler(repo,collections=collections)

def makeRegistry(butler):
    '''
    Small function to convert a Butler obj to a registry shim
    '''
    return butler.registry

def makeKwargs(run_list,img_type,detector_num,instrument='LSSTCam'):
    '''
    A function to make the kwargs in a butler.registry.QueryDatasets command
    '''
    return {"datasetType":"{}".format(img_type),"where":"exposure.science_program in ({}) and instrument='{}' and detector={}".format(makeRunNums(run_list),instrument,detector_num)}

def makeRunNums(run_list):
    '''
    A function to format the run list from [12345,67890] format to the format needed for the butler query
    '''
    return str(list(map(str, run_list)))[1:-1]

def fetchAllRefs(repo,collections,run_list,img_type,detector_num,instrument='LSSTCam'):
    '''
    A function to fetch all refs meeting the criteria of the above
    '''
    butler = makeButler(repo,collections)
    registry = makeRegistry(butler)
    kwargs = makeKwargs(run_list,img_type,detector_num,instrument=instrument)
    return getRefList(registry,kwargs)

def refSelector(refList,criteria,butler,false_criteria={},onlyUnique=True):
    '''
    A function to filter out refs within a refList, based on criteria (dict)
    '''
    outRefs = []
    for ref in refList:
        passCriteria = True
        exp = butler.get(ref)
        mData = exp.getInfo().getMetadata().toDict()
        for key,val in zip(criteria.keys(),criteria.values()):
            if mData[key]!=val:
                passCriteria=False
                break
        for key,val in zip(false_criteria.keys(),false_criteria.values()):
            if mData[key]==val:
                passCriteria=False
                break
        if passCriteria:
            outRefs.append(ref)
    if onlyUnique:
        return np.unique(outRefs)
    else:
        return outRefs

def getMetadataDict(ref,butler):
    '''
    Function to get metadata of a ref as a dict
    '''
    return butler.get(ref).getInfo().getMetadata().toDict()

def retrieveImage(repo,collection,run_list,img_type,detector_num,criteria):
    '''
    A function to return an ExposureF based on criteria, repo, collection, run_list, etc....
    '''
    refs = fetchAllRefs(repo,collection,run_list,img_type,detector_num)
    filtered_refs = refSelector(refs,criteria,makeButler(repo,collection))
    if len(filtered_refs)!=1:
        return makeButler(repo,collection).get(filtered_refs[0]) # Probably would be good to have a better way to select, but this works for now I guess.....
    else:
        return makeButler(repo,collection).get(filtered_refs)

In [4]:
run_list = [13330,14524,13550]
img_type="raw"
detector_num = 100
repo='/repo/ir2'
collection= 'LSSTCam/raw/all,LSSTCam/calib'.split(",")
criteria = {'IMGTYPE': 'FLAT',"EXPTIME":15}

In [214]:
refs = fetchAllRefs(repo,collection,run_list,img_type,detector_num)

In [215]:
exp = makeButler(repo,collection).get(refs[0])

In [5]:
image = retrieveImage(repo,collection,run_list,img_type,detector_num,criteria)